진호님이 일단 실행되게끔 만들어 본것

In [6]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')
train_text[:100]
a= train_text.split('\n')[1:][0]

a.split('\t')[2]
# X 값 추출 = 리뷰문장 == 피쳐
x_train = [ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0]
x_train[:3]
#y값 추출  == target
y_train= np.array([ [int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)
y_train
from konlpy.tag import Okt  #트위터에서 만든 한국어분석 메서드
from konlpy.tag import Kkma  #서울대에서 만든거 morphs== 형태소 분석


x_tokenized = [ ]
for i in x_train[:100]:
    x_tokenized.append( Okt().morphs( i ) ) 
    
x_tokenized[:5]
# . ! ? ;;; 은 는 이 가 => 불용어 제거
# 불용어 빈도수가 늘어남 = 중요도 판별하는 순위가 되기 때문에
# => TF-IDF  TF= 빈도수  IDF = 역 빈도수 

# 0 부정문들만 모여있는 문서
# 1 긍정문들만 모여있는 문서

# => TF-IDF 알고리즘은 두 문서에 겹치는 단어들 동시에 많이 쓰이는 단어들은 점수를 낮게
# => 겹치지않는 단어들은 점수를 높게 매겨준다.
import pandas as pd

x_train[:100] #1차원


y_train[:100]  #2차원 

x_df = pd.DataFrame(x_train[:10000])
# print(x_df)

y_df = pd.DataFrame(y_train[:10000])
# print(y_df)

# pd.DataFrame(x_df, y_df, columns=['x','y'], axis= 1)

df = pd.concat([x_df, y_df], axis=1)

# df
# df.rename(columns={0, 0 : 'x','y'})
# df_1= df[ (df.iloc[1:] ==0) ] 


bad = df[df.iloc[:,1]==0]
good = df[df.iloc[:,1]==1]

bad
# TF-IDF 단어 중요도 점수를 매긴다.


# ======================
# bad[:3]
good[:3]
test111=pd.concat([bad,good],axis=0) #라벨이 같이 들어온다.
test111


from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
vectorizer = TfidfVectorizer( tokenizer= Okt().morphs ) # 매개변수를 조정하여 다양한 옵션을 적용할 수 있습니다.
X = vectorizer.fit_transform(test111.iloc[:,0])
y = test111.iloc[:,1]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# bad_text = ''
# for i in bad.iloc[:,0]:
#     bad_text += i
    
# # print(bad_text)

# good_text = ''
# for i in good.iloc[:,0]:
#     good_text += i
    
# # print(good_text)

# texts =[ bad_text , good_text ]
# texts

# from konlpy.tag import Kkma
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# vectorizer = CountVectorizer(tokenizer= Kkma().morphs )

# # 요쯤에 불용어 제거 들어가야할것같은 느낌 => 생략
    
# X = vectorizer.fit_transform( texts )

# word_list = vectorizer.get_feature_names_out()
# # word_list

# # 오탈자, 잘 쓰지 않는말 같은경우는 가중치가 높아질수있다.
# print(X.toarray())  #TF본다. 각문서에서의 빈도수를 확인한다.
# idf = TfidfTransformer()
# X_tfidf = idf.fit_transform( X )

# print( X_tfidf.toarray( ) )
# len(X_tfidf.toarray()[0])

# 딥러닝 모델

# y_train=np.array([
#     [0],
#     [1]
# ])


# X_tfidf.toarray()


(150000, 1)


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [7]:
import numpy as np

X_train_dense = np.expand_dims(X_train.toarray(), axis=-1)
X_test_dense = np.expand_dims(X_test.toarray(), axis=-1)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

# 모델 구성
model = Sequential([
    Input(shape=(X_train_dense.shape[1],)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_dense.squeeze(), y_train, epochs=10, batch_size=32, validation_split=0.2)

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
200/200 [==============================] - 12s 51ms/step - loss: 0.5811 - accuracy: 0.7534 - val_loss: 0.4435 - val_accuracy: 0.8062
Epoch 2/10
200/200 [==============================] - 11s 55ms/step - loss: 0.2700 - accuracy: 0.9167 - val_loss: 0.3958 - val_accuracy: 0.8194
Epoch 3/10
200/200 [==============================] - 10s 50ms/step - loss: 0.1188 - accuracy: 0.9742 - val_loss: 0.4058 - val_accuracy: 0.8175
Epoch 4/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0599 - accuracy: 0.9898 - val_loss: 0.4296 - val_accuracy: 0.8181
Epoch 5/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0346 - accuracy: 0.9958 - val_loss: 0.4601 - val_accuracy: 0.8131
Epoch 6/10
200/200 [==============================] - 10s 52ms/step - loss: 0.0225 - accuracy: 0.9970 - val_loss: 0.4826 - val_accuracy: 0.8094
Epoch 7/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0159 - accuracy: 0.9980 - val_loss: 0.5040 - val_accuracy:

In [4]:
!pip install konlpy


In [20]:
test = ["비가 오니까 약속을 취소해야 되는게 너무 싫다."]

# 테스트 데이터를 전처리 (옵션, 예: 소문자로 변환, 불용어 제거 등)
# 여기에 전처리 코드를 추가할 수 있습니다.

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round(predictions).astype(int))

1/1 [==============================] - 0s 74ms/step
예측 결과: [[0]]


In [ ]:
model.save('기본 DNN 적용모델.h5')